In [1]:
query = 'How do I execute a command in a running docker container?'

In [2]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

In [3]:
index_name = "course-questions"

In [5]:
def check_indexed_data(index_name):
    search_query = {
        "size": 10,
        "query": {
            "match_all": {}
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    return response

indexed_data = check_indexed_data(index_name)
print(indexed_data)


{'took': 2, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 948, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'course-questions', '_id': 'klwuWZABHMPiqGEh8ysj', '_score': 1.0, '_source': {'text': 'What are equivalents in Scikit-Learn for the linear regression with and without regularization used in week 2.\nCorresponding function for model without regularization:\nsklearn.linear_model.LinearRegression\nCorresponding function for model with regularization:\nsklearn.linear_model.Ridge\nThe linear model from Scikit-Learn are explained  here:\nhttps://scikit-learn.org/stable/modules/linear_model.html\nAdded by Sylvia Schmitt', 'section': '2. Machine Learning for Regression', 'question': 'Corresponding Scikit-Learn functions for Linear Regression (with and without Regularization)', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'k1wuWZABHMPiqGEh8ysw', '_score': 1.0, '_source': {'

In [6]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"], #question is 3 times (boost) more important that text or question
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    for hit in response['hits']['hits']:
        print(hit['_score'])

In [7]:
elastic_search(query)

84.050095
75.54128
72.08518
51.04628
49.938507


In [8]:
def check_course_field_values(index_name):
    search_query = {
        "size": 10,
        "query": {
            "match_all": {}
        },
        "_source": ["course"]
    }
    response = es_client.search(index=index_name, body=search_query)
    return response

course_values = check_course_field_values(index_name)
print(course_values)


{'took': 3, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 948, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'course-questions', '_id': 'klwuWZABHMPiqGEh8ysj', '_score': 1.0, '_source': {'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'k1wuWZABHMPiqGEh8ysw', '_score': 1.0, '_source': {'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'lFwuWZABHMPiqGEh8ys4', '_score': 1.0, '_source': {'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'lVwuWZABHMPiqGEh8ytY', '_score': 1.0, '_source': {'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'llwuWZABHMPiqGEh8ytg', '_score': 1.0, '_source': {'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'l1wuWZABHMPiqGEh8yto', '_score': 1.0, '_source': {'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'm

In [15]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"], #question is 3 times (boost) more important that text or question
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [17]:
results = elastic_search(query)

In [18]:
results

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [19]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [42]:
context = ""

for doc in results:
    context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"

prompt = context_template.format(question=query, text=context).strip()

In [43]:
context

'Q: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani\n\nQ: How do I copy files from a different folder into docker container’s working directory?\nA: You can copy files from your local machine into a Docker container using the docker cp comman

In [44]:
prompt

'Q: How do I execute a command in a running docker container?\nA: Q: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani\n\nQ: How do I copy files from a different folder into docker container’s working directory?\nA: You can copy files from your

In [45]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [50]:
prompt_len = prompt_template.format(question="How do I execute a command in a running docker container?", 
                           context=context).strip()
len(prompt_len)

1462

In [47]:
import tiktoken

In [48]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [56]:
encoding.encode(prompt_len)

[63842,
 261,
 4165,
 14029,
 29186,
 13,
 30985,
 290,
 150339,
 4122,
 402,
 290,
 31810,
 8099,
 591,
 290,
 40251,
 7862,
 558,
 8470,
 1606,
 290,
 19719,
 591,
 290,
 31810,
 8099,
 1261,
 55959,
 290,
 150339,
 364,
 107036,
 25,
 3253,
 621,
 357,
 15792,
 261,
 6348,
 306,
 261,
 6788,
 62275,
 9282,
 1715,
 10637,
 50738,
 734,
 48,
 25,
 3253,
 621,
 357,
 15199,
 261,
 62275,
 9282,
 3901,
 32,
 25,
 41281,
 290,
 9282,
 3621,
 306,
 25383,
 6766,
 326,
 151187,
 290,
 7251,
 4859,
 11,
 813,
 484,
 480,
 13217,
 261,
 38615,
 6348,
 558,
 68923,
 2461,
 533,
 278,
 2230,
 7962,
 4859,
 38615,
 464,
 3365,
 523,
 3335,
 290,
 9282,
 382,
 4279,
 6788,
 11,
 15792,
 261,
 6348,
 306,
 290,
 4857,
 9282,
 734,
 68923,
 10942,
 350,
 6555,
 290,
 9282,
 26240,
 446,
 68923,
 25398,
 533,
 278,
 464,
 6896,
 26240,
 29,
 38615,
 198,
 6103,
 277,
 10732,
 391,
 79771,
 1029,
 48,
 25,
 3253,
 621,
 357,
 5150,
 6291,
 591,
 922,
 2698,
 7342,
 316,
 62275,
 9282,
 3901,
 32,
 2

In [52]:
len(encoding.encode(prompt_len))

322

In [53]:
encoding.decode_single_token_bytes(63842)

b"You're"

In [54]:
from openai import OpenAI
def llm(prompt):
    with open('../open-aikey.txt', 'r') as file:
        key = file.read().strip()
    client = OpenAI(api_key=key)
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role":"user", "content":prompt}]
    )
    return response.choices[0].message.content

In [55]:
llm(prompt)

'You can copy files from a different folder on your local machine into a Docker container’s working directory using the `docker cp` command. Here\'s how you can achieve this:\n\n1. **Using docker cp Command:**\n\n   To copy files or directories from a specific folder on your local machine into the working directory of a running Docker container, you can use the `docker cp` command.\n\n   Suppose you have the following folder structure on your local machine:\n\n   ```\n   /path/to/local/source/\n       - file1.txt\n       - file2.txt\n       - subdirectory/\n           - file3.txt\n   ```\n\n   And you want to copy these files into the working directory of a Docker container:\n\n   ```sh\n   docker cp /path/to/local/source/file1.txt <container_id>:/path/to/container/working_directory/\n   docker cp /path/to/local/source/subdirectory <container_id>:/path/to/container/working_directory/\n   ```\n\n   If you\'re not sure about the container\'s working directory, you can first find it using

- input/request: 0.00074
- output/request: 0.00375
- price/request: 0.0045
- 1000 requests: 4.5$